In [162]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
q


ValueError: ignored

In [227]:
# Imports

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
from IPython.display import HTML
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import codecs

In [228]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [229]:
# Full width display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# TASK 1

In [230]:
# Load data
train_df = pd.read_csv('train.csv', dtype={'grades': 'string'})
train_df['grades'] = train_df['grades'].str.replace('', ' ').str.strip()

#test_df = pd.read_csv('data/task-1/dev.csv')
#test_df['grades'] = test_df['grades'].str.replace('', ' ').str.strip()

In [231]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9652 entries, 0 to 9651
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         9652 non-null   int64  
 1   original   9652 non-null   object 
 2   edit       9652 non-null   object 
 3   grades     9652 non-null   string 
 4   meanGrade  9652 non-null   float64
dtypes: float64(1), int64(1), object(2), string(1)
memory usage: 377.2+ KB


In [232]:
#test_df.info()

In [233]:
print(train_df['original'][0])
print()
train_df.head()

France is ‘ hunting down its citizens who joined <Isis/> ’ without trial in Iraq



,id,original,edit,grades,meanGrade
0,14530,France is ‘ hunting down its citizens who joined <Isis/> ’ without trial in Iraq,twins,1 0 0 0 0,0.2
1,13034,"Pentagon claims 2,000 % increase in Russian trolls after <Syria/> strikes . What does that mean ?",bowling,3 3 1 1 0,1.6
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor Crashes <Coalition/>,party,2 2 1 0 0,1.0
3,76,"In an apparent first , Iran and Israel <engage/> each other militarily",slap,2 0 0 0 0,0.4
4,6164,Trump was told weeks ago that Flynn misled <Vice/> President .,school,0 0 0 0 0,0.0


In [236]:
# Expanding the stupidly jointed grades

grade_names = []
for i in range(15):
    grade_names.append(f'grade {i+1}')
train_df[grade_names] = train_df['grades'].str.split(' ', expand=True)

# Turn valid grades to int, missing ones to NaN
train_df[grade_names] = train_df[grade_names].apply(pd.to_numeric, errors='coerce')

In [237]:
train_df.head()

# Whole df as html 
# HTML(train_df.to_html())

,id,original,edit,grades,meanGrade,grade 1,grade 2,grade 3,grade 4,grade 5,grade 6,grade 7,grade 8,grade 9,grade 10,grade 11,grade 12,grade 13,grade 14,grade 15
0,14530,France is ‘ hunting down its citizens who joined <Isis/> ’ without trial in Iraq,twins,1 0 0 0 0,0.2,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13034,"Pentagon claims 2,000 % increase in Russian trolls after <Syria/> strikes . What does that mean ?",bowling,3 3 1 1 0,1.6,3,3,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8731,Iceland PM Calls Snap Vote as Pedophile Furor Crashes <Coalition/>,party,2 2 1 0 0,1.0,2,2,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,76,"In an apparent first , Iran and Israel <engage/> each other militarily",slap,2 0 0 0 0,0.4,2,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6164,Trump was told weeks ago that Flynn misled <Vice/> President .,school,0 0 0 0 0,0.0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9652 entries, 0 to 9651
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         9652 non-null   int64  
 1   original   9652 non-null   object 
 2   edit       9652 non-null   object 
 3   grades     9652 non-null   string 
 4   meanGrade  9652 non-null   float64
 5   grade 1    9652 non-null   int64  
 6   grade 2    9652 non-null   int64  
 7   grade 3    9652 non-null   int64  
 8   grade 4    9652 non-null   int64  
 9   grade 5    9652 non-null   int64  
 10  grade 6    142 non-null    float64
 11  grade 7    142 non-null    float64
 12  grade 8    142 non-null    float64
 13  grade 9    142 non-null    float64
 14  grade 10   142 non-null    float64
 15  grade 11   6 non-null      float64
 16  grade 12   6 non-null      float64
 17  grade 13   6 non-null      float64
 18  grade 14   6 non-null      float64
 19  grade 15   6 non-null      float64
dtypes: float

How to best handle the 6 with 15 grads and the 142 with 10 grades?

In [190]:
train_df.sort_values('meanGrade',ascending=False)[['original','edit', 'meanGrade']]

,original,edit,meanGrade
6075,Recent Scandals Highlight Trump 's Chaotic <Management/> Style,Fashion,3.0
1490,"President Trump 's first year <anniversary/> report card , with grades from A + to F",Kindergarten,3.0
4438,How an FBI raid fed a rumor that Orrin Hatch was about to become <president/>,pregnant,2.8
8726,Trump admits tariffs could cause ' pain ' in <markets/>,buttock,2.8
6946,Kasich : Trump <tweets/> ‘ unacceptable ’,hair,2.8
...,...,...,...
9250,It did n’t end at the ballot box : <Donald Trump/> ’s biggest supporters now push for a divisive agenda,cutlery,0.0
1237,Delhi Police Say They 've Captured Most-Wanted Terrorist Known As ' <India/> 's Bin Laden ',Hindu,0.0
9253,Trump 's Climate-Denying Coal Lobbyist Nominee Inches Closer To EPA ’s No. 2 <Job/>,Mine,0.0
4421,Democratic division simmers at feel-good <retreat/>,stove,0.0


# TASK 2

In [ ]:
# Load data
train_df = pd.read_csv('data/task-2/train.csv')
test_df = pd.read_csv('data/task-2/dev.csv')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9381 entries, 0 to 9380
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9381 non-null   object 
 1   original1   9381 non-null   object 
 2   edit1       9381 non-null   object 
 3   grades1     9381 non-null   int64  
 4   meanGrade1  9381 non-null   float64
 5   original2   9381 non-null   object 
 6   edit2       9381 non-null   object 
 7   grades2     9381 non-null   int64  
 8   meanGrade2  9381 non-null   float64
 9   label       9381 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 733.0+ KB


In [ ]:
train_df

,id,original1,edit1,grades1,meanGrade1,original2,edit2,grades2,meanGrade2,label
0,10920-9866,""" Gene Cernan , Last <Astronaut/> on the Moon ...",Dancer,1113,1.2,""" Gene Cernan , Last Astronaut on the Moon , <...",impregnated,30001,0.8,1
1,3176-10722,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,0
2,3176-3702,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
3,10722-3702,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
4,12282-2083,""" Our expectations of what civic engagement lo...",imagine,0,0.0,""" Our expectations of what civic engagement <l...",smells,100220010,0.6,2
...,...,...,...,...,...,...,...,...,...,...
9376,975-13357,"“ It ’s painfully obvious "" Mueller will charg...",battery,1,0.2,"“ It ’s painfully obvious "" Mueller will charg...",plumbing,11103,1.2,2
9377,975-11773,"“ It ’s painfully obvious "" Mueller will charg...",battery,1,0.2,"“ It ’s painfully obvious "" Mueller will <char...",strangle,22331,2.2,2
9378,13357-11773,"“ It ’s painfully obvious "" Mueller will charg...",plumbing,11103,1.2,"“ It ’s painfully obvious "" Mueller will <char...",strangle,22331,2.2,2
9379,14954-14479,"“ Kompromat , ” media ethics and the law : Wha...",porn,20101,0.8,"“ Kompromat , ” media ethics and the law : Wha...",dance,32112,1.8,2


In [ ]:
funniest = train_df.sort_values('meanGrade1',ascending=False)[['original1','edit1','meanGrade1','edit2']]

In [ ]:
for i in range(5):
    print(funniest['original1'][i],funniest['edit1'][i],' vs ',funniest['edit2'][i])

" Gene Cernan , Last <Astronaut/> on the Moon , Dies at 82 " Dancer  vs  impregnated
" I 'm done " : Fed up with California , some <conservatives/> look to Texas vagrants  vs  pancakes
" I 'm done " : Fed up with California , some <conservatives/> look to Texas vagrants  vs  life
" I 'm done " : Fed up with <California/> , some conservatives look to Texas pancakes  vs  life
" Our expectations of what civic engagement looks like do n’t match reality . Can we <fix/> that ? " imagine  vs  smells
